In [13]:
import os
from scipy.io import loadmat

folder_path = r"..\los_data"  # 你的文件夹路径
file_list = [f for f in os.listdir(folder_path) if f.endswith('.mat')]  # 找到所有mat文件
file_list = sorted(file_list)  # 按文件名排序，比如 dev0.mat, dev1.mat, dev2.mat

data = []  # 用来存储每个mat文件中的 'data_Ineed'（转置后的）

for i, file_name in enumerate(file_list):
    full_path = os.path.join(folder_path, file_name)
    mat_contents = loadmat(full_path)
    if 'data_Ineed' in mat_contents:
        data_ineed = mat_contents['data_Ineed'].T  # 加上 .T 转置！
        data.append(data_ineed)
    else:
        print(f"Warning: {file_name} 里没有 'data_Ineed' 变量！")

# 打印检查
for i, d in enumerate(data):
    print(f"第{i}个文件（{file_list[i]}）的数据 shape (转置后): {d.shape},信号数量*采样点数量")



第0个文件（dev0.mat）的数据 shape (转置后): (7014, 320),信号数量*采样点数量
第1个文件（dev1.mat）的数据 shape (转置后): (6632, 320),信号数量*采样点数量
第2个文件（dev10.mat）的数据 shape (转置后): (8935, 320),信号数量*采样点数量
第3个文件（dev11.mat）的数据 shape (转置后): (7776, 320),信号数量*采样点数量
第4个文件（dev13.mat）的数据 shape (转置后): (7410, 320),信号数量*采样点数量
第5个文件（dev14.mat）的数据 shape (转置后): (6274, 320),信号数量*采样点数量
第6个文件（dev15.mat）的数据 shape (转置后): (5706, 320),信号数量*采样点数量
第7个文件（dev16.mat）的数据 shape (转置后): (12968, 320),信号数量*采样点数量
第8个文件（dev17.mat）的数据 shape (转置后): (9508, 320),信号数量*采样点数量
第9个文件（dev18.mat）的数据 shape (转置后): (3343, 320),信号数量*采样点数量
第10个文件（dev19.mat）的数据 shape (转置后): (4756, 320),信号数量*采样点数量
第11个文件（dev2.mat）的数据 shape (转置后): (6316, 320),信号数量*采样点数量
第12个文件（dev20.mat）的数据 shape (转置后): (5592, 320),信号数量*采样点数量
第13个文件（dev3.mat）的数据 shape (转置后): (5536, 320),信号数量*采样点数量
第14个文件（dev4.mat）的数据 shape (转置后): (6669, 320),信号数量*采样点数量
第15个文件（dev5.mat）的数据 shape (转置后): (5237, 320),信号数量*采样点数量
第16个文件（dev6.mat）的数据 shape (转置后): (5460, 320),信号数量*采样点数量
第17个文件（dev7.mat）的数据 shape (转置后): (8926, 320),信号

In [14]:
import numpy as np

# 假设data是从之前的加载代码中获取的
# data = <从前一部分代码中加载的数据>

SNR_dB = -5  # 设定信噪比（dB）

noisy_data = []  # 用来存储加噪声后的数据

for d in data:
    # 计算信号的标准差
    signal_std = np.std(d)
    
    # 计算噪声的标准差，根据信噪比（SNR）
    noise_std = signal_std / (10 ** (SNR_dB / 20))  # 根据 SNR(dB) 计算噪声标准差
    
    # 生成高斯噪声并加到原信号上
    noise = np.random.normal(0, noise_std, d.shape)  # 均值为0，标准差为噪声标准差
    noisy_signal = d + noise  # 添加噪声
    
    noisy_data.append(noisy_signal)  # 保存加噪声后的信号

# 打印检查加噪声后的数据
print("原始数据和加噪声数据示例：")
print(f"原始信号样本（第一个信号）：{data[0][0][:10]}")  # 打印前10个采样点
print(f"加噪声信号样本（第一个信号）：{noisy_data[0][0][:10]}")  # 打印前10个采样点


原始数据和加噪声数据示例：
原始信号样本（第一个信号）：[-0.01318091-0.00598181j -0.01379628-0.00492906j -0.017992  +0.00138218j
 -0.185444  +0.24452379j -0.6173177 +0.97452922j -0.15240179+0.98618152j
  0.87295696-0.30563645j  0.19726927-0.65624198j  0.0687799 +0.41792528j
  0.95404651+0.17878775j]
加噪声信号样本（第一个信号）：[ 3.74881214-0.00598181j  2.06829455-0.00492906j  0.25251821+0.00138218j
 -1.07041813+0.24452379j  0.12427222+0.97452922j -1.62815961+0.98618152j
 -0.87517828-0.30563645j -2.01626637-0.65624198j  0.94204299+0.41792528j
  3.74811586+0.17878775j]


In [15]:
import numpy as np

def compute_doppler_shift(v, fc):
    """
    根据移动速度 v (m/s) 和载波频率 fc (Hz) 计算多普勒频移
    """
    c = 3e8  # 光速 m/s
    return (v / c) * fc

def add_doppler_shift(signal, fd, fs):
    """
    给IQ信号加多普勒频移
    signal: shape (num_channels, num_samples)，复数IQ信号
    fd: 多普勒频移 (Hz)
    fs: 采样率 (Hz)
    """
    num_channels, num_samples = signal.shape
    t = np.arange(num_samples) / fs  # 时间轴
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)  # 复指数
    return signal * doppler_phase

# 例子
fs = 20e6  # 采样率，比如1 MHz
fc = 2.4e9  # 载波频率，比如2.4 GHz Wi-Fi
v = 120  # 移动速度，比如30 m/s (~108 km/h)

fd = compute_doppler_shift(v, fc)  # 先计算多普勒频移
print(f"计算得到的多普勒频移 fd = {fd:.2f} Hz")

# 假设 data 是你之前读出来的list，每个是 shape (信号数量, 采样点数)
data_with_doppler = []
for sig in noisy_data:
    shifted_sig = add_doppler_shift(sig, fd, fs)
    data_with_doppler.append(shifted_sig)

print("添加多普勒频移后的信号：")
print(data_with_doppler)


计算得到的多普勒频移 fd = 960.00 Hz
添加多普勒频移后的信号：
[array([[ 3.74881214e+00-5.98180779e-03j,  2.06829594e+00-4.30527379e-03j,
         2.52517326e-01+1.53449983e-03j, ...,
         6.16731635e-01+6.75318288e-01j,  3.27355550e-01-7.04980969e-01j,
        -3.92648820e+00-3.64734284e-01j],
       [-1.01799322e-01-6.65819948e-03j,  4.36414274e+00-4.27605597e-03j,
        -3.26687118e-03+1.52807518e-04j, ...,
        -2.64102767e+00-3.57220641e-01j,  9.47558271e-01-1.18668016e+00j,
        -3.14965105e+00-6.96952542e-01j],
       [ 2.46781758e-01+3.14398823e-03j,  5.80010395e-02+1.51891490e-03j,
         5.23593074e-01-2.31765039e-03j, ...,
         1.14441777e+00+6.31893096e-01j,  1.72214619e+00+1.51174057e+00j,
         2.88924855e+00+7.39552301e-01j],
       ...,
       [-1.09362723e+00+1.49593753e-03j,  1.25556622e+00+1.24025271e-03j,
        -1.97488206e-01+2.43261231e-02j, ...,
         1.92333070e+00+1.23435051e+00j,  4.86867564e-01+1.26972653e+00j,
         1.69051407e+00+5.16242580e-01j],
    

In [16]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from datetime import datetime
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

# =================== 设备检查 ===================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# =================== 数据准备 ===================
# 假设 data_with_doppler 是你的列表，每个元素 shape = (样本数, 320)
all_data = []
all_labels = []

for device_idx, device_signals in enumerate(data_with_doppler):
    device_labels = np.full((device_signals.shape[0],), device_idx)
    all_data.append(torch.tensor(device_signals, dtype=torch.float32))
    all_labels.append(torch.tensor(device_labels, dtype=torch.long))

all_data = torch.cat(all_data, dim=0)
all_labels = torch.cat(all_labels, dim=0)

dataset = TensorDataset(all_data, all_labels)

# ------------------- 划分训练/验证/测试集 -------------------
test_ratio = 0.2
val_ratio = 0.2

test_size = int(len(dataset) * test_ratio)
train_val_size = len(dataset) - test_size

train_val_dataset, test_dataset = random_split(dataset, [train_val_size, test_size])

train_size = int(train_val_size * (1 - val_ratio))
val_size = train_val_size - train_size

train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# ==== 数据增强：对训练集添加高斯噪声 ====
def add_noise_to_dataset(dataset, noise_std=0.01):
    noisy_data = []
    noisy_labels = []
    for x, y in dataset:
        noise = torch.randn_like(x) * noise_std
        noisy_x = x + noise
        noisy_data.append(noisy_x)
        noisy_labels.append(y)
    noisy_data = torch.stack(noisy_data)
    noisy_labels = torch.tensor(noisy_labels)
    return TensorDataset(noisy_data, noisy_labels)

train_dataset = add_noise_to_dataset(train_dataset, noise_std=0.01)

# =================== Transformer 模型定义 ===================
class SignalTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        self.transformer = nn.Transformer(
            d_model=model_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc = nn.Linear(model_dim, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x)
        if len(x.shape) == 3:
            x = x[:, -1, :]
        x = self.fc(x)
        return x

# =================== 参数空间 ===================
param_space = {
    "model_dim": [64, 128, 256],     # 新增
    "num_heads": [2, 4, 8],
    "num_layers": [1, 2, 3],
    "dropout": [0.1, 0.3, 0.5],
    "learning_rate": [1e-3, 5e-4, 1e-4],
    "weight_decay": [1e-4, 1e-5],
    "batch_size": [64, 128, 256]
}

num_classes = 20
input_dim = 320
num_trials = 100
num_epoch = 300

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
script_name = "time_random"
folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{fd}_classes_{num_classes}_Transformer"
save_folder = os.path.join(os.getcwd(), "search_results", folder_name)
os.makedirs(save_folder, exist_ok=True)
results_file = os.path.join(save_folder, "results.txt")

trial_results = []
best_result = {"val_acc": 0}

# =================== 训练+验证+测试流程 ===================
def train_and_evaluate(hparams):
    batch_size = hparams["batch_size"]
    lr = hparams["learning_rate"]
    wd = hparams["weight_decay"]

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

    model = SignalTransformer(
        input_dim=input_dim,
        model_dim=hparams["model_dim"],  # 使用 model_dim
        num_heads=hparams["num_heads"],
        num_layers=hparams["num_layers"],
        num_classes=num_classes,
        dropout=hparams["dropout"]
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    patience, patience_counter = 8, 0
    best_val_loss = float("inf")

    for epoch in range(num_epoch):
        # ===== 训练阶段 =====
        model.train()
        train_correct, train_total = 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (preds == labels).sum().item()

        train_acc = 100 * train_correct / train_total

        # ===== 验证阶段 =====
        val_loss, val_correct, val_total = 0, 0, 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()

        val_acc = 100 * val_correct / val_total

        # 早停检查
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
            best_train_acc = train_acc  # 记录当时的训练集准确率
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    # ===== 测试阶段 =====
    model.load_state_dict(best_model_state)
    test_correct, test_total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (preds == labels).sum().item()

    test_acc = 100 * test_correct / test_total

    return {
        "train_acc": best_train_acc,   # 新增
        "val_loss": best_val_loss / len(val_loader),
        "val_acc": val_acc,
        "test_acc": test_acc,
        "best_model_state": best_model_state
    }

# =================== 随机搜索循环 ===================
for trial in range(num_trials):
    hparams = {k: random.choice(v) for k, v in param_space.items()}
    print(f"\n=== Trial {trial+1}/{num_trials} ===")
    print(hparams)

    metrics = train_and_evaluate(hparams)

    record = {**hparams,
              "train_acc": metrics["train_acc"],  # 新增
              "val_acc": metrics["val_acc"],
              "test_acc": metrics["test_acc"]}
    trial_results.append(record)

    print(f"Train Acc: {metrics['train_acc']:.2f}%, "
          f"Validation Acc: {metrics['val_acc']:.2f}%, "
          f"Test Acc: {metrics['test_acc']:.2f}%")

    with open(results_file, "a") as f:
        f.write(f"Trial {trial+1}: {record}\n")

    if metrics["val_acc"] > best_result.get("val_acc", 0):
        best_result = {**record}
        best_model_path = os.path.join(save_folder, "best_model_state.pth")
        torch.save(metrics["best_model_state"], best_model_path)


with open(results_file, "a") as f:
    f.write("\n=== Best Result ===\n")
    f.write(str(best_result) + "\n")

print("Random search finished.")
print("Best params:", best_result)

# =================== 混淆矩阵绘制 ===================
def plot_confusion_matrix(best_model_state, best_hparams):
    batch_size = best_hparams["batch_size"]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

    model = SignalTransformer(
        input_dim=input_dim,
        model_dim=best_hparams["model_dim"],    # 适配 model_dim
        num_heads=best_hparams["num_heads"],
        num_layers=best_hparams["num_layers"],
        num_classes=num_classes,
        dropout=best_hparams["dropout"]
    ).to(device)
    model.load_state_dict(best_model_state)
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(num_classes))
    plt.figure(figsize=(12, 10))
    disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
    plt.title("Confusion Matrix - Best Model")
    cm_path = os.path.join(save_folder, "best_model_confusion_matrix.png")
    plt.savefig(cm_path, dpi=300)
    plt.close()
    print(f"Confusion matrix saved to: {cm_path}")

# 调用混淆矩阵绘制（注意这里传入保存的 best_model_state）
plot_confusion_matrix(torch.load(best_model_path), best_result)

Using device: cuda

=== Trial 1/100 ===
{'model_dim': 256, 'num_heads': 8, 'num_layers': 1, 'dropout': 0.5, 'learning_rate': 0.0005, 'weight_decay': 0.0001, 'batch_size': 256}
Train Acc: 21.66%, Validation Acc: 20.04%, Test Acc: 19.89%

=== Trial 2/100 ===
{'model_dim': 64, 'num_heads': 2, 'num_layers': 1, 'dropout': 0.1, 'learning_rate': 0.001, 'weight_decay': 0.0001, 'batch_size': 128}
Train Acc: 22.07%, Validation Acc: 20.54%, Test Acc: 20.04%

=== Trial 3/100 ===
{'model_dim': 64, 'num_heads': 2, 'num_layers': 1, 'dropout': 0.1, 'learning_rate': 0.001, 'weight_decay': 0.0001, 'batch_size': 128}
Train Acc: 22.34%, Validation Acc: 20.49%, Test Acc: 20.03%

=== Trial 4/100 ===
{'model_dim': 128, 'num_heads': 8, 'num_layers': 2, 'dropout': 0.3, 'learning_rate': 0.001, 'weight_decay': 1e-05, 'batch_size': 256}
Train Acc: 9.30%, Validation Acc: 9.49%, Test Acc: 9.32%

=== Trial 5/100 ===
{'model_dim': 256, 'num_heads': 4, 'num_layers': 3, 'dropout': 0.1, 'learning_rate': 0.0005, 'weight_